## List the docker image to verify the image is built

In [6]:
! docker images

REPOSITORY            TAG            IMAGE ID       CREATED          SIZE
zoomcamp-model        3.13.10-hw10   a814dda000f1   19 seconds ago   567MB
clothing-classifier   v1             1e73cc6cb121   22 hours ago     524MB
kindest/node          <none>         452d707d4862   3 weeks ago      1.31GB
redis                 latest         f0957bcaa75f   3 months ago     222MB
mysql                 latest         94254b456a6d   4 months ago     1.27GB


## Question 1

In [7]:
!docker ps

CONTAINER ID   IMAGE                         COMMAND                  CREATED         STATUS         PORTS                                         NAMES
4705c346565a   zoomcamp-model:3.13.10-hw10   "uvicorn q6_predict:…"   8 seconds ago   Up 8 seconds   0.0.0.0:9696->9696/tcp, [::]:9696->9696/tcp   elastic_shtern


In [10]:
!ls q6_test.py

q6_test.py


In [11]:
!python q6_test.py

{'conversion_probability': 0.49999999999842815, 'conversion': False}


> > Answer : 0.49

## Question 2

### verify kind and kubectl installation

In [12]:
! kind version

kind v0.31.0 go1.25.5 darwin/arm64


In [13]:
! kubectl version --client

Client Version: v1.35.0
Kustomize Version: v5.7.1


In [23]:
! kind create cluster --name mlzoomcamp-week10-hw

Creating cluster "mlzoomcamp-week10-hw" ...
 ✓ Ensuring node image (kindest/node:v1.35.0) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-mlzoomcamp-week10-hw"
You can now use your cluster with:

kubectl cluster-info --context kind-mlzoomcamp-week10-hw

Thanks for using kind! 😊


In [24]:
!kubectl cluster-info
!kubectl get nodes

Kubernetes control plane is running at https://127.0.0.1:56684
CoreDNS is running at https://127.0.0.1:56684/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
NAME                                 STATUS     ROLES           AGE   VERSION
mlzoomcamp-week10-hw-control-plane   NotReady   control-plane   6s    v1.35.0


> Answer : kind version v0.31.0

## Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?

> Answer : **Pod**

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the Type of the service that is already running there?
- NodePort
- ClusterIP
- ExternalName
- LoadBalancer

In [25]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   9s


> Answer : **ClusterIP**

**Notes** : The kubernetes service (which is the default Kubernetes API server service that's automatically created in every cluster) has a TYPE of ClusterIP. This is the default service type in Kubernetes that makes the service accessible only within the cluster on an internal IP.


## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.13.10-hw10`), we need to register it with kind.

What's the command we need to run for that?
- `kind create cluster`
- `kind build node-image`
- `kind load docker-image`
- `kubectl apply`

In [26]:
!kind load docker-image zoomcamp-model:3.13.10-hw10 --name mlzoomcamp-week10-hw

Image: "zoomcamp-model:3.13.10-hw10" with ID "sha256:a814dda000f17180eb5bf67406bedc6b2f1a0c824494cc1b8b1080124b20a8a5" not yet present on node "mlzoomcamp-week10-hw-control-plane", loading...


> Answer : `kind load docker-image`

## Question 6

Now let's create a deployment config (e.g. deployment.yaml):

```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [17]:
! cat hw/deployment.yml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.13.10-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"
          limits:
            memory: "256Mi"
            cpu: "500m"
        ports:
        - containerPort: 9696

> Answer : 9696

In [27]:
! kubectl apply -f hw/deployment.yml

deployment.apps/subscription created


In [29]:
! kubectl get pods

NAME                           READY   STATUS    RESTARTS   AGE
subscription-7b98b7548-tlkl9   1/1     Running   0          5s


## Question 7

Let's create a service for this deployment (`service.yaml`):

```
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?
Apply this config file.

In [30]:
! cat hw/service.yml

apiVersion: v1
kind: Service
metadata:
  name: subscription-service
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696

In [31]:
! kubectl apply -f hw/service.yml

service/subscription-service created


In [32]:
! kubectl get services

NAME                   TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes             ClusterIP      10.96.0.1       <none>        443/TCP        3m52s
subscription-service   LoadBalancer   10.96.249.186   <pending>     80:31309/TCP   17s


In [35]:
!python q6_test.py

{'conversion_probability': 0.49999999999842815, 'conversion': False}


> Answer : subscription

## Question 8

Run kubectl get hpa subscription-hpa --watch command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test?

- 1
- 2
- 3
- 4

> **Note**: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from initial.


Prerequisites: Install Metrics Server in your kind cluster.

In [39]:
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


Patch the metrics server to work in kind cluster

In [59]:
!kubectl patch -n kube-system deployment metrics-server --type=json -p '[{"op":"add","path":"/spec/template/spec/containers/0/args/-","value":"--kubelet-insecure-tls"}]'

deployment.apps/metrics-server patched


8.1. Create HPA

In [61]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu='20%' --min=1 --max=3

horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


8.2. List the hpa and verify it's created

In [62]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: 3%/20%   1         3         1          3s


In [63]:
!cat q6_load.py

import requests
from time import sleep

url = "http://localhost:9696/predict"

client = {"job": "management", "duration": 400, "poutcome": "success"}
while True:
    sleep(0.05)
    response = requests.post(url, json=client).json()
    print(response)


Note : q6_load.py is running on the background.

In [66]:
!kubectl get hpa subscription-hpa

NAME               REFERENCE                 TARGETS        MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: 61%/20%   1         3         3          97s


> Answer : 3 (replicas)